# Part B 

In [1]:
import sys
import random
from pprint import pprint as pp
random.seed(42)
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [2]:
import neo4j
import py2neo
print(neo4j.__version__)
print(py2neo.__version__)

1.7.6
4.3.0


In [3]:
from neo4j import GraphDatabase

# instantiate driver
NEO4J_URI="bolt://localhost:7687"
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

## H-index

From wikipedia

> The h-index is defined as the maximum value of h such that the given author/journal has published h papers that have each been cited at least h times.[6] The index is designed to improve upon simpler measures such as the total number of citations or publications.[citation needed] The index works properly only for comparing scientists working in the same field; citation conventions differ widely among different fields.[citation needed]
>
> Formally, if f is the function that corresponds to the number of citations for each publication, we compute the h-index as follows. First we order the values of f from the largest to the lowest value. Then, we look for the last position in which f is greater than or equal to the position (we call h this position). For example, if we have a researcher with 5 publications A, B, C, D, and E with 10, 8, 5, 4, and 3 citations, respectively, the h-index is equal to 4 because the 4th publication has 4 citations and the 5th has only 3. In contrast, if the same publications have 25, 8, 5, 3, and 3 citations, then the index is 3 because the fourth paper has only 3 citations.
>
>        f(A)=10, f(B)=8, f(C)=5, f(D)=4, f(E)=3　→ h-index=4
>        f(A)=25, f(B)=8, f(C)=5, f(D)=3, f(E)=3　→ h-index=3
>
> If we have the function f ordered in decreasing order from the largest value to the lowest one, we can compute the h-index as follows:
>
>    $$\text{h-index} (f) = {\displaystyle \max _{i}\min(f(i),i)}$$

- obtain $f$, the number of citations of each article
- order $f$ from largest to lowest
- from the last element in f

The citation count can be obtained by matching a pattern

```cypher
(a2:article)<-[:cited_by]-(a1:article)-[:authored_by]->(p:author)
```

and counting the number of articles with alias `a2`.

In [5]:
q1 = """MATCH p=(a2:article)<-[:cited_by]-(a1:article)-[:authored_by]->(author:author)
WITH a1, author, COUNT(a2) as citation_count
ORDER BY a1, author, citation_count
RETURN a1, author, citation_count
LIMIT 5"""

with gdb.session() as session:
    _out = session.run(q1).values()
    
print(_out)

[[<Node id=2470781 labels={'article'} properties={'ee': ['https://doi.org/10.1002/asi.20428'], 'volume': '57', 'number': '11', 'journal': 'JASIST', 'pages': '1519-1525', 'mdate': neotime.Date(2017, 5, 28), 'year': 2006, 'author': ['Ido Dagan', 'Moshe Koppel', 'Navot Akiva'], 'title': 'Feature instability as a criterion for selecting potential style markers.', 'article': 37408, 'key': 'journals/jasis/KoppelAD06', 'url': 'db/journals/jasis/jasis57.html#KoppelAD06'}>, <Node id=4787441 labels={'author'} properties={'affiliation_institution_name': 'Merritt, Rivas and Harrison', 'affiliation_institution_type': 'NGO'}>, 3], [<Node id=2470781 labels={'article'} properties={'ee': ['https://doi.org/10.1002/asi.20428'], 'volume': '57', 'number': '11', 'journal': 'JASIST', 'pages': '1519-1525', 'mdate': neotime.Date(2017, 5, 28), 'year': 2006, 'author': ['Ido Dagan', 'Moshe Koppel', 'Navot Akiva'], 'title': 'Feature instability as a criterion for selecting potential style markers.', 'article': 374

In [6]:
WITH a, collect(number_of_cites) as list_of_cites
WITH a, [i in range(0, size(list_of_cites) - 1) |
i + 1 <= list_of_cites[i]] as cites_boolean
RETURN a, size(filter(b in cites_boolean WHERE b));

SyntaxError: invalid syntax (<ipython-input-6-31757566f314>, line 1)